In [9]:
%matplotlib qt
import xarray as xr
from matplotlib import colors
from matplotlib.animation import FuncAnimation, writers, PillowWriter
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cmocean
import cartopy.crs as ccrs
import cartopy
import numpy as np
import copy

Load in particle data, define the time range at which to plot (**`plottimes`**) and select the indices of the first timestep in the variable `'b'`. Then load the OceanParcels logo.

In [19]:
filename = 'medusarun.nc'
pfile = xr.open_dataset(str(filename), decode_cf=True)
lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)

pfile.close()

plottimes = np.arange(time[0,0], np.nanmax(time), np.timedelta64(10, 'D'))
starttime = 20
b = time == plottimes[0+starttime]

img = mpimg.imread('parcelslogo.png')

The animation consists of three figures: the northern hemisphere, the southern hemisphere and the oceanparcels logo. To organize their locations we use [matplotlib.gridspec](https://matplotlib.org/tutorials/intermediate/gridspec.html). The animation spans 12 frames and updates the particle positions based on the timestep in `plottimes`.

In [20]:
fig = plt.figure(figsize=(20,4))
gs = gridspec.GridSpec(ncols=60, nrows=12, figure=fig)

### Northern Hemisphere
ax1 = fig.add_subplot(gs[:, 18:30],projection=ccrs.NearsidePerspective(central_latitude=90, central_longitude=-30, satellite_height=15000000))
ax1.set_facecolor('#190C38')
ax1.add_feature(cartopy.feature.LAND,zorder=1)
ax1.coastlines()
scat1 = ax1.scatter(lon[b], lat[b], marker='.', s=25, c='maroon',edgecolor='white', linewidth=0.15, transform=ccrs.PlateCarree())

### Southern Hemisphere
ax2 = fig.add_subplot(gs[:, 30:42],projection=ccrs.NearsidePerspective(central_latitude=-90, central_longitude=-30, satellite_height=15000000))
ax2.set_facecolor('#190C38')
ax2.add_feature(cartopy.feature.LAND,zorder=1)
ax2.coastlines()
scat2 = ax2.scatter(lon[b], lat[b], marker='.', s=25, c='maroon',edgecolor='white', linewidth=0.15, transform=ccrs.PlateCarree())

### OceanParcels logo
ax3 = fig.add_subplot(gs[1:3,27:33])
ax3.tick_params(
    axis='both',         
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    left=False,
    labelleft=False) # labels along the bottom edge are off
ax3.imshow(img)
ax3.spines["left"].set_visible(False)
ax3.spines["top"].set_visible(False)
ax3.spines["right"].set_visible(False)
ax3.spines["bottom"].set_visible(False)

frames = np.arange(0, 12)

def animate(t):
    b = time == plottimes[t+start]
    scat1.set_offsets(np.vstack((lon[b], lat[b])).transpose())
    scat2.set_offsets(np.vstack((lon[b], lat[b])).transpose())
    return scat1, scat2

anim = animation.FuncAnimation(fig, animate, frames=frames, interval=200, blit=True)
anim
fig.canvas.draw()  # needed for tight_layout to work with cartopy
plt.tight_layout()
# writergif = PillowWriter(fps=7) 
# anim.save('Homepagegif.gif',writer=writergif)